### Create student csv file in the form

|id  | sex | entrance_year | grad_class | final_major         | final_major_main | final_major_sub | code_semester           |
|----|-----|---------------|------------|---------------------|------------------|-----------------|-------------------------|
|000 | F/M | 2003          | 2007       | Engineering Robotics| Engineering      | Robotics        | ['FND1210 0304FA', ...] |

In [4]:
import pandas as pd
data = pd.read_csv('../data/olincourseenrollment0214.csv')

In [5]:
data.head()

,status,grad_class,id,semester,sex,grade,major_main,major_sub,code,Unnamed: 9,course_main,course_sub,professor,term,year,course_fullname,major_fullname,code-semester
0,GRAD,2007,602,0304FA,M,FF,Undeclared,NaN,AHS1101,2,History and Society,Bodies in Motion: Migration...,"Fleischmann, Fritz",FA,3,History and Society Bodies in Motion: Migratio...,Undeclared,AHS1101 0304FA
1,GRAD,2007,602,0304FA,M,FF,Undeclared,NaN,FND1210,1,Physical Foundations ofEngineering I,NaN,"Zastavker, Yevgeniya",FA,3,Physical Foundations ofEngineering I,Undeclared,FND1210 0304FA
2,GRAD,2007,602,0304FA,M,FF,Undeclared,NaN,FND1312,1,Mathematical Foundations ofEngineering I: Calc...,NaN,"Tilley, Burt S.",FA,3,Mathematical Foundations ofEngineering I: Calc...,Undeclared,FND1312 0304FA
3,GRAD,2007,602,0304FA,M,FF,Undeclared,NaN,FND1410,1,Foundations of EngineeringProject I,Mechanical Nature,"Linder, Benjamin",FA,3,Foundations of EngineeringProject I Mechanical...,Undeclared,FND1410 0304FA
4,GRAD,2007,602,0304FA,M,FF,Undeclared,NaN,FND1510,3,Intro to Modeling & Control,NaN,"Pratt, Gill",FA,3,Intro to Modeling & Control,Undeclared,FND1510 0304FA


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29537 entries, 0 to 29536
Data columns (total 18 columns):
status             29537 non-null object
grad_class         29537 non-null int64
id                 29537 non-null int64
semester           29537 non-null object
sex                29537 non-null object
grade              29537 non-null object
major_main         29537 non-null object
major_sub          10997 non-null object
code               29537 non-null object
Unnamed: 9         29537 non-null object
course_main        29537 non-null object
course_sub         4386 non-null object
professor          29537 non-null object
term               29537 non-null object
year               29537 non-null int64
course_fullname    29537 non-null object
major_fullname     29537 non-null object
code-semester      29537 non-null object
dtypes: int64(3), object(15)
memory usage: 4.3+ MB


In [20]:
def semester_year(semester):
    year = 2000 + int(semester[0:2])
    return year

data['semester_year'] = data['semester'].apply(semester_year)
data.head()

,status,grad_class,id,semester,sex,grade,major_main,major_sub,code,Unnamed: 9,course_main,course_sub,professor,term,year,course_fullname,major_fullname,code-semester,semester_year
0,GRAD,2007,602,0304FA,M,FF,Undeclared,NaN,AHS1101,2,History and Society,Bodies in Motion: Migration...,"Fleischmann, Fritz",FA,3,History and Society Bodies in Motion: Migratio...,Undeclared,AHS1101 0304FA,2003
1,GRAD,2007,602,0304FA,M,FF,Undeclared,NaN,FND1210,1,Physical Foundations ofEngineering I,NaN,"Zastavker, Yevgeniya",FA,3,Physical Foundations ofEngineering I,Undeclared,FND1210 0304FA,2003
2,GRAD,2007,602,0304FA,M,FF,Undeclared,NaN,FND1312,1,Mathematical Foundations ofEngineering I: Calc...,NaN,"Tilley, Burt S.",FA,3,Mathematical Foundations ofEngineering I: Calc...,Undeclared,FND1312 0304FA,2003
3,GRAD,2007,602,0304FA,M,FF,Undeclared,NaN,FND1410,1,Foundations of EngineeringProject I,Mechanical Nature,"Linder, Benjamin",FA,3,Foundations of EngineeringProject I Mechanical...,Undeclared,FND1410 0304FA,2003
4,GRAD,2007,602,0304FA,M,FF,Undeclared,NaN,FND1510,3,Intro to Modeling & Control,NaN,"Pratt, Gill",FA,3,Intro to Modeling & Control,Undeclared,FND1510 0304FA,2003


In [59]:
studentd = {'id':[],
            'sex':[], 
            'entrance_year':[], 
            'grad_class': [], 
            'final_major': [], 
            'final_major_main': [], 
            'final_major_sub': [], 
            'code-semester':[]}

tag = 0
studenttag = -1
for i in data.index:
    if data['id'][i] not in studentd['id']:
        if tag == i-1:
            studentd['grad_class'].append(data['grad_class'][i-1])
            studentd['final_major'].append(data['major_fullname'][i-1])
            studentd['final_major_main'].append(data['major_main'][i-1])
            studentd['final_major_sub'].append(data['major_sub'][i-1])
        else:
            studentd['id'].append(data['id'][i])
            studentd['sex'].append(data['sex'][i])
            studentd['entrance_year'].append(data['semester_year'][i])
            studentd['code-semester'].append([data['code-semester'][i]])
            studenttag += 1
    else:
        tag = i
        studentd['code-semester'][studenttag].append(data['code-semester'][i])

studentd['grad_class'].append(data['grad_class'][len(data)-1])
studentd['final_major'].append(data['major_fullname'][len(data)-1])
studentd['final_major_main'].append(data['major_main'][len(data)-1])
studentd['final_major_sub'].append(data['major_sub'][len(data)-1])

In [60]:
student = pd.DataFrame(studentd)

In [63]:
columns = ['id','sex','entrance_year','grad_class','final_major','final_major_main','final_major_sub','code-semester']
student = student[columns]
student.head(10)

,id,sex,entrance_year,grad_class,final_major,final_major_main,final_major_sub,code-semester
0,602,M,2003,2007,Mechanical Engineering,Mechanical Engineering,NaN,"[AHS1101 0304FA, FND1210 0304FA, FND1312 0304F..."
1,537,F,2002,0,Engineering,Engineering,NaN,"[FND1210 0203FA, FND1310 0203FA, FND1311 0203F..."
2,856,M,2003,2007,Engineering Systems,Engineering,Systems,"[FND1210 0304FA, FND1312 0304FA, FND1410 0304F..."
3,535,F,2004,2008,Engineering Computing,Engineering,Computing,"[ENGR1110 0405FA, ENGR1200 0405FA, MTH1110 040..."
4,483,M,2004,2008,Engineering Systems,Engineering,Systems,"[ENGR1110 0405FA, ENGR1200 0405FA, MTH1110 040..."
5,559,F,2004,2008,Engineering Materials Science,Engineering,Materials Science,"[ENGR1110 0405FA, ENGR1200 0405FA, ENGR1510 04..."
6,861,M,2003,2007,Mechanical Engineering,Mechanical Engineering,NaN,"[FND1210 0304FA, FND1312 0304FA, FND1410 0304F..."
7,853,M,2004,2008,Engineering Computing,Engineering,Computing,"[ENGR1110 0405FA, ENGR1200 0405FA, MTH1110 040..."
8,625,M,2002,2006,Engineering,Engineering,NaN,"[FND1210 0203FA, FND1310 0203FA, FND1311 0203F..."
9,123,F,2003,2007,Engineering Bioengineering,Engineering,Bioengineering,"[FND1210 0304FA, FND1312 0304FA, FND1410 0304F..."


In [65]:
student.to_csv('../data/student0214.csv', index = False)